# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [4]:
!docker run -d --name nginx --network=none -p 81:81 nginx

388e8ffc503c9e9681646cd3c47f2017e3fa889d2dcfd3ff3a893946ce2601f6


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [28]:
!curl localhost

#the curl command should fail on our system 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 80 after 2230 ms: Could not connect to server


Lets run it on the docker container 

In [29]:
!docker exec -it nginx /bin/sh -c "curl google.com"



Error response from daemon: No such container: nginx


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [30]:
!docker network

Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [31]:
!docker network ls 

NETWORK ID     NAME          DRIVER    SCOPE
a43a50685720   bridge        bridge    local
34ae92ce7788   host          host      local
c8520680d4c9   none          null      local
b9cecc94cbf2   yourNetwork   bridge    local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [32]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "c8520680d4c9dd395b39d9d495666a362d8f215fa6fb01c5e37766413acd8239",
        "Created": "2025-03-15T03:09:24.175983955Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [10]:
!pip install docker

Defaulting to user installation because normal site-packages is not writeable
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
Using cached docker-7.1.0-py3-none-any.whl (147 kB)


In [38]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Docker version: 28.0.1


Lets create a quick code to list the containers using the docker client

In [36]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

adminer - running
mysql - running


Here you can see the running container and no surprise it is the nginx we started before


In [37]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


NotFound: 404 Client Error for http+docker://localnpipe/v1.48/containers/nginx/json: Not Found ("No such container: nginx")

again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [39]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

docker: Error response from daemon: Conflict. The container name "/mysql" is already in use by container "71d71b1c884768d2cfdba0f3a3ebed45b13a58a64937c2ac3a6828918bc3c34c". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


## The following needs to be run in the terminal

In [40]:
!docker run -d --name adminer -p 8080:8080 adminer


docker: Error response from daemon: Conflict. The container name "/adminer" is already in use by container "3b3dbb87aeabfd0c3e1c6efbe78c372597062c9d7f90958576546ad3714cfd14". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [41]:
#!docker inspect mysql | grep -n1 "IPAddress\":"
# or
#print("-"*30)
show_container_ip("mysql")


mysql in network 'yourNetwork' has IP: 172.18.0.2


In [42]:
#!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
#print("-"*30)
show_container_ip("adminer")


adminer in network 'yourNetwork' has IP: 172.18.0.3


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [56]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        container.reload()
        if container.status == 'exited':
            print(f"Remove {container.name}")
            container.remove()

list_running_containers()

clean_containers()


adminer - running
mysql - running
Remove adminer
Remove mysql


## Setting up the network

In [44]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

Error response from daemon: network with name yourNetwork already exists


In [45]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "b9cecc94cbf21d556b76768b69d4a60a408efc651c1c9b475d1e55818db62a8e",
        "Created": "2025-04-19T04:41:57.666642215Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [46]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


34d5e617036ccd8458b5a7746e79cf97c3eb6de9aa3f2733a961dc0dd8178cdb


In [47]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

f4a1893875b73c2035ce686e0bc5ebd808a3a871528de0d620389f4d39f3b361


In [48]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [49]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

'grep' is not recognized as an internal or external command,
operable program or batch file.


------------------------------
mysql in network 'yourNetwork' has IP: 172.18.0.2


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [50]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

'grep' is not recognized as an internal or external command,
operable program or batch file.
'grep' is not recognized as an internal or external command,
operable program or batch file.


Access denied. Option -c requires administrative privileges.
------------------------------
adminer in network 'yourNetwork' has IP: 172.18.0.3


# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 



In [51]:

import docker


client = docker.from_env()
last_restart_times = {}


def get_running_containers():
    return client.containers.list()
def get_all_containers():
    return client.containers.list(all=True)

def get_container_ips(container_name):
    container = client.containers.get(container_name)
    status = container.status
    
    if status == "running":
        networks = container.attrs['NetworkSettings']['Networks']
        for net_name, net_data in networks.items():
            print(f"{container_name} is running in network '{net_name}' with IP: {net_data['IPAddress']}") 
    else:
        print(f"{container_name} is not running and has status: {status}")

for container in get_running_containers():
    get_container_ips(container.name)





adminer is running in network 'yourNetwork' with IP: 172.18.0.3
mysql is running in network 'yourNetwork' with IP: 172.18.0.2



## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/



In [ ]:
import datetime


def monitor_and_restart():
    containers = get_all_containers()
    for container in containers:
        container.reload()
        if container.status != 'running':
            print(f"Restarting {container.name} (Status: {container.status})")
            try:
                container.restart()
            except Exception as e:
                print(f"Error restarting {container.name}: {e}")

def maintenace_restart():
    containers = get_all_containers()
    for container in containers:
        now = datetime.datetime.now()
        last_restart = last_restart_times.get(container.name)
        if not last_restart or (now - last_restart).total_seconds()> 86400:
            print(f"Performing schedyules restart for {container.name}")
            try:
                container.restart()
                last_restart_times[container.name]=now
            except Exception as e:
                print(f"Error during maintance retsart for {container.name}: e")
                

 

In [55]:
monitor_and_restart()
maintenace_restart()

Performing schedyules restart for adminer
Performing schedyules restart for mysql
